# Creación de los corpus para pruebas del corrector ortográfico

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import re

language = "ash"

Inicialización del dataframe, eliminación de espacios blancos multiples, eliminación de espacios en blanco al inicio y final de las oraciones; y cambiar valores del dataframe vacíos a NaN

In [2]:
df = pd.read_excel("trabajo.xlsx", "Modernización", skiprows=9, usecols="B:E", names=["number", "word" , "sentence", "sentence_unnormalized"])

df = df.replace({"\s+": " "}, regex=True)
df = df.apply(lambda row: row.str.strip() if row.dtype == "object" else row)
df["sentence"] = df["sentence"].str.lower()
df["sentence_unnormalized"] = df["sentence_unnormalized"].str.lower()
df = df.replace(r"^\s*$", np.nan, regex=True)

In [3]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 800


Remover las filas que no tengan una oración y que no tengan ningún tipo de error etiquetado

In [4]:
df = df[df["sentence"].notna()]
df = df[df["sentence_unnormalized"].notna()]

In [5]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 800


Identificar los caracteres únicos de las oraciones para propósitos de limpieza manual del excel

In [6]:
chars = sorted(list(set((df["sentence"] + df["sentence_unnormalized"]).sum())))
print("Lista de caracteres (ambos conjuntos de oraciones combinadas): {}".format(chars))

Lista de caracteres (ambos conjuntos de oraciones combinadas): [' ', "'", ',', '.', 'a', 'b', 'c', 'e', 'h', 'i', 'j', 'k', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'y', 'ñ']


Agregar las columnas "words" y "words_with_errors" para identificar las palabras en las oraciones que tienen errores ortográficos, también identificaremos las filas en donde las oraciones no coinciden en número de palabras

In [7]:
df["words"] = df["words_unnormalized"] = None
allowed_chars_regex = "[^0-9A-ZÁÉÍÓÚÑa-záéíóúñ\s\'\-]+"
differente_words_count_idxs = []

for _, row in df.iterrows():
    sentence = re.sub(allowed_chars_regex, '', row["sentence"])
    sentence_with_errors = re.sub(allowed_chars_regex, '', row["sentence_unnormalized"])
    
    sentence_word_list = sentence.strip().split(' ')
    sentence_with_errors_word_list = sentence_with_errors.strip().split(' ')    
    
    if(len(sentence_word_list) != len(sentence_with_errors_word_list)):
        differente_words_count_idxs.append(row["number"])
        continue
    
    errors_count = 0
    words = []
    word_errors = []
    for idx, word in enumerate(sentence_word_list):        
        if word != sentence_with_errors_word_list[idx]:
            errors_count += 1
            words.append(word)
            word_errors.append(sentence_with_errors_word_list[idx])
    
    df.loc[df["number"] == row["number"], "sentence"] = sentence
    df.loc[df["number"] == row["number"], "sentence_unnormalized"] = sentence_with_errors
    df.loc[df["number"] == row["number"], "words"] = ','.join(words)
    df.loc[df["number"] == row["number"], "words_unnormalized"] = ','.join(word_errors)

In [8]:
print("{} filas en donde el número de palabras de la oración sin errores ortográficos no es igual al número de palabras de la oracion con errores ortográficos: {}".format(len(differente_words_count_idxs), differente_words_count_idxs))

0 filas en donde el número de palabras de la oración sin errores ortográficos no es igual al número de palabras de la oracion con errores ortográficos: []


Remover las filas en donde las oraciones no coinciden en número de palabras

In [9]:
df = df[~df["number"].isin(differente_words_count_idxs)]

In [10]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 800


Agregar la columna "error_types_count" al dataframe que indique el número de "error_types"

In [11]:
df["word_unnormalized_count"] = df.apply(lambda row: len(row["words_unnormalized"].split(',')) if row["words_unnormalized"] else 0, axis=1)

Remover las filas en donde no tengan ninguna palabra con errores ortográficos.

In [12]:
df = df[~(df["word_unnormalized_count"] == 0)]

In [13]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 796


In [14]:
df.head()

,number,word,sentence,sentence_unnormalized,words,words_unnormalized,word_unnormalized_count
0,1,irako,ipesakero irako koki ipesabetaro shibitsa abot...,ipesaquero iraco coqui ipesavetaro shivitsa av...,"ipesakero,irako,koki,ipesabetaro,shibitsa,abot...","ipesaquero,iraco,coqui,ipesavetaro,shivitsa,av...",6
1,2,onkitaitamane,ari namenamanakeri onkitaitamane ibankoki pink...,ari namanamanakeri onquitaitamane ivancoqui pi...,"namenamanakeri,onkitaitamane,ibankoki,pinkatsari","namanamanakeri,onquitaitamane,ivancoqui,pincat...",4
2,3,tsompoikira,pamenaaranteri tsompoinakira imaita paitarika ...,pamenaaranteri tsompoinaquira imaita paitarica...,"tsompoinakira,paitarika,yobayetiri,ikoshitiri","tsompoinaquira,paitarica,yovayetiri,icoshitiri",4
3,4,amitakotantyariri,pikamantena narori amitakotantyaariri pomerent...,pikamantena narori amitacotantyariri pomerents...,"amitakotantyaariri,pomerentsitimotakariri,pichane","amitacotantyariri,pomerentsitimotacariri,picha...",3
4,5,pamineri,yamakeri atiri pameneri piyotanteariri iriotak...,yamaqueri atiri pameneri piyotanteariri iriota...,"yamakeri,iriotake,koshitantatsiri","yamaqueri,iriotaque,coshitantasiri",3


In [15]:
print("Número de oraciones sin errores ortográficos y con errores ortográficos (cada uno): {}".format(df.shape[0]))

Número de oraciones sin errores ortográficos y con errores ortográficos (cada uno): 796


In [16]:
sentences = df["sentence"].tolist()
corpus = ' '.join(sentences)
words = [w.strip() for w in corpus.split()]
word_count = Counter(words)
unique_words = list(word_count.keys())

print("Número de palabras únicas (oraciones sin errores ortográficos): {}".format(len(unique_words)))

Número de palabras únicas (oraciones sin errores ortográficos): 3124


In [17]:
chars = sorted(list(set(df["sentence"].sum())))
print("{} caracteres (oraciones sin errores ortográficos): {}".format(len(chars), chars))

22 caracteres (oraciones sin errores ortográficos): [' ', "'", 'a', 'b', 'c', 'e', 'h', 'i', 'j', 'k', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'y', 'ñ']


In [18]:
sentences = df["sentence_unnormalized"].tolist()
corpus = ' '.join(sentences)
words = [w.strip() for w in corpus.split()]
word_count = Counter(words)
unique_words = list(word_count.keys())

print("Número de palabras únicas (oraciones con errores ortográficos): {}".format(len(unique_words)))

Número de palabras únicas (oraciones con errores ortográficos): 3291


In [19]:
chars = sorted(list(set(df["sentence_unnormalized"].sum())))
print("{} caracteres (oraciones con errores ortográficos): {}".format(len(chars), chars))

22 caracteres (oraciones con errores ortográficos): [' ', "'", 'a', 'b', 'c', 'e', 'h', 'i', 'j', 'k', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'y', 'ñ']


Crear corpus de oraciones sin errores ortograficos

In [20]:
with open(language + ".teacher_unnorm.test.sentences.txt", "w") as f:
    lines = (df["sentence"] + '\n').tolist()
    f.writelines(lines)

with open(language + ".teacher_unnorm.test.sentences.errors.txt", "w") as f:
    lines = (df["sentence_unnormalized"] + '\n').tolist()
    f.writelines(lines)